# Assignment 

## Suppose you are a police department with a limited budget. You plan to  reduce road-traffic accidents by a one-month targeted advertising campaign. Using the given dataset, which gender, age group, and month would be the largest target group as indicated by positive breath tests?

### Introduction
------------------
##### Using the given dataset DigitalBreathTestData(2013), we have to analyze the data set and come up with the conclusion
1. Which month the police department has to plan the advertisement campaign. An example of which months recorded the most number of road accidents due to drunk and drive.
1. Which age-group and gender are mainly involved in road accidents. 

##### Based on these data output, the police department will focus the advertisement campaign on the exact group of civilians and aware them.

To generate insight, I am using my jupyter notebook. I would be using the Markdown cells to explain the code and referencing.
#### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------
Environment: pyspark, jupyter notebook.

DataSet: DigitalBreathTest(2013)

In [2]:
import pyspark

from pyspark import SparkContext, SparkConf
conf = SparkConf().setMaster("local").setAppName("breathtest")
#sc = SparkContext(conf = conf)
sc = SparkContext.getOrCreate()


##### Check we have a spark context: Should show the spark version and app name

#### Constructs a connection to Spark execution environment

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

#### Loading file from local file system, using sc.textfile(), and checking wether file loaded properly by using lines.take()

In [4]:
lines = sc.textFile("file:///Users/alanj/Desktop/DigitalBreathTestData2013.txt")
lines.take(5)

Py4JJavaError: An error occurred while calling o38.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/Users/alanj/Desktop/DigitalBreathTestData2013.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:205)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:276)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:272)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:276)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:272)
	at org.apache.spark.api.java.JavaRDDLike.partit

#### Extracting column header / removing the first row which contains the column values

In [23]:
from itertools import islice

lines = lines.mapPartitionsWithIndex(
    lambda idx, it: islice(it, 1, None) if idx == 0 else it 
)

#### First split the file into columns, and just use the first column ('Reason') as the key

In [9]:
def parseline(line):
    fields = line.split(',')
    Reason = fields[0]
    return (Reason, 1)

#### Apply the parseline function as an parameter to the map function

In [24]:
allreasons = lines.map(parseline)

#### Count total lines in the file

In [25]:
allreasons.count()

497790

#### Count the different reason types

In [12]:
allreasons.countByKey()

defaultdict(int,
            {'Moving Traffic Violation': 179064,
             'Road Traffic Collision': 168526,
             'Suspicion of Alcohol': 94685,
             'Other': 55372,
             'Unknown': 143})

#### Inputting the attribute value line into parseline2 function

In [26]:
def parseline2(line):
    fields = line.split(',')
    Reason = fields[0]
    Month = fields[1]
    Year = fields[2]
    WeekType = fields[3]
    TimeBand = fields[4]
    BreathAlcoholLevel = int(fields[5])
    AgeBand = fields[6]
    if fields[7]=='Female':
        Gender = 2
    else:
        Gender = 1
    return (Reason,Month,Year,WeekType,TimeBand,BreathAlcoholLevel,AgeBand,Gender)


In [27]:
rdd = lines.map(parseline2)
rdd.count()

497790

#### generate new RDD in which BreathAlcoholLevel > 0 (Needs to exceed 35 to be illegal in UK, 80mg for blood)

In [1]:
filtered_reasons = rdd.filter(lambda x: x[5] > 35)

NameError: name 'rdd' is not defined

In [29]:
filtered_reasons.count()

52658

In [30]:
sorted(filtered_reasons.countByKey().items())

[('Moving Traffic Violation', 11336),
 ('Other', 7971),
 ('Road Traffic Collision', 16641),
 ('Suspicion of Alcohol', 16702),
 ('Unknown', 8)]

In [31]:
rddInverted = filtered_reasons.map(lambda x: (x[4],1)).reduceByKey(lambda x, y: x + y)
sorted(rddInverted.collect())

[('12am-4am', 17998),
 ('12pm-4pm', 4054),
 ('4am-8am', 6134),
 ('4pm-8pm', 6855),
 ('8am-12pm', 4452),
 ('8pm-12pm', 13161),
 ('Unknown', 4)]

#### From below result we can find that most number of alcahol suspision accidents happened between the 12am to 4am, the number of cases are 31090

In [32]:
timebandands= rddInverted.sortBy(lambda x: x[1], ascending=False).collect()
for result in timebandands:
    print(result[0],': ', result[1])

12am-4am :  17998
8pm-12pm :  13161
4pm-8pm :  6855
4am-8am :  6134
8am-12pm :  4452
12pm-4pm :  4054
Unknown :  4


In [33]:
rddInverted_month = filtered_reasons.map(lambda x: (x[1],1)).reduceByKey(lambda x, y: x + y)
sorted(rddInverted_month.collect())

[('Apr', 4223),
 ('Aug', 4919),
 ('Dec', 5268),
 ('Feb', 3261),
 ('Jan', 3093),
 ('Jul', 4693),
 ('Jun', 5299),
 ('Mar', 4443),
 ('May', 4993),
 ('Nov', 4285),
 ('Oct', 4137),
 ('Sep', 4044)]

#### From the below result, we can conclude that most numbers of alcohol suspicion road accidents cases reported in December month, According to that police department can start the awareness campaign on December. PFB

In [34]:
Month = rddInverted_month.sortBy(lambda x: x[1], ascending=False).collect()
for result in Month:
    print(result[0],': ', result[1])

Jun :  5299
Dec :  5268
May :  4993
Aug :  4919
Jul :  4693
Mar :  4443
Nov :  4285
Apr :  4223
Oct :  4137
Sep :  4044
Feb :  3261
Jan :  3093


In [36]:
rddInverted_gender = filtered_reasons.map(lambda x: (x[7],1)).reduceByKey(lambda x, y: x + y)
sorted(rddInverted_gender.collect())

[(1, 44045), (2, 8613)]

In [37]:
Gender = rddInverted_gender.sortBy(lambda x: x[1], ascending=False).collect()
for result in Gender:
    print(result[0],': ', result[1])

1 :  44045
2 :  8613


#### According to the above result, most number of alcahol suspicion accidensts reported from males. So the most vulnurable and targeted gender is male.

In [39]:
rddInverted_AgeBand = filtered_reasons.map(lambda x: (x[6],1)).reduceByKey(lambda x, y: x + y)
sorted(rddInverted_AgeBand.collect())


[('16-19', 2486),
 ('20-24', 8192),
 ('25-29', 7381),
 ('30-39', 11186),
 ('40-49', 8615),
 ('50-59', 4892),
 ('60-69', 2032),
 ('70-98', 550),
 ('Other', 7324)]

In [40]:
AgeBand = rddInverted_AgeBand.sortBy(lambda x: x[1], ascending=False).collect()
for result in AgeBand:
    print(result[0],': ', result[1])


30-39 :  11186
40-49 :  8615
20-24 :  8192
25-29 :  7381
Other :  7324
50-59 :  4892
16-19 :  2486
60-69 :  2032
70-98 :  550


#### According to the above result, The targeted age group is 30-39. But it will be great if campaign covers age group >= 18

#### Based on the above analysis the main findings are..
  1. The most number of alcahol suspicion accidensts reported on month of December (12161 cases)
  2. The age group 30-39 are reported most number of cases compare to other age groups